# LCEL Interface

- Author: [JeongGi Park](https://github.com/jeongkpa)
- Peer Review: [YooKyung Jeon](https://github.com/sirena1), [Wooseok Jeong](https://github.com/jeong-wooseok)
- Proofread : [Q0211](https://github.com/Q0211)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/01-Basic/07-LCEL-Interface.ipynb)
[![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/01-Basic/07-LCEL-Interface.ipynb)

## Overview

The LangChain Expression Language (LCEL) is a powerful interface designed to simplify the creation and management of custom chains in LangChain. 
It implements the Runnable protocol, providing a standardized way to build and execute language model chains.


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [LCEL Runnable Protocol](#lcel-runnable-protocol)
- [stream: real-time output](#stream-real-time-output)
- [Invoke](#invoke)
- [batch: unit execution](#batch-unit-execution)
- [async stream](#async-stream)
- [async invoke](#async-invoke)
- [async batch](#async-batch)
- [Parallel](#parallel)
- [Parallelism in batches](#parallelism-in-batches)

### References

- [Langsmith DOC](https://docs.smith.langchain.com/)
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- ```langchain-opentutorial``` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [```langchain-opentutorial```](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

set environment variables is in .env.

Copy the contents of .env_sample and load it into your .env with the key you set.

In [34]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [9]:
%%capture --no-stderr
!pip install langchain-opentutorial


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain-openai",
        "langchain",
        "python-dotenv",
        "langchain-core",
    ],
    verbose=False,
    upgrade=False,
)

In [35]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        # "OPENAI_API_KEY": "<Your OpenAI API KEY>",
        # "LANGCHAIN_API_KEY": "<Your LangChain API KEY>",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "LangSmith-Tracking-Setup",  # title 과 동일하게 설정해 주세요
    }
)

Environment variables have been set successfully.


## LCEL Runnable Protocol

---

To make it as easy as possible to create custom chains, we've implemented the ```Runnable``` protocol.

The ```Runnable``` protocol is implemented in most components.

It is a standard interface that makes it easy to define custom chains and call them in a standard way. The standard interface includes

- ```stream```: Streams a chunk of the response.
- ```invoke```: Invoke a chain on an input.
- ```batch```: Invoke a chain against a list of inputs.

There are also asynchronous methods

- ```astream```: Stream chunks of the response asynchronously.
- ```ainvoke```: Invoke a chain asynchronously on an input.
- ```abatch```: Asynchronously invoke a chain against a list of inputs.
- ```astream_log```: Streams the final response as well as intermediate steps as they occur.



### Log your trace

We provide multiple ways to log traces to LangSmith. Below, we'll highlight how to use traceable().

Use the code below to record a trace in LangSmith

In [36]:
import openai
from langsmith import wrappers, traceable

# Auto-trace LLM calls in-context
client = wrappers.wrap_openai(openai.Client())

@traceable # Auto-trace this function
def pipeline(user_input: str):
    result = client.chat.completions.create(
        messages=[{"role": "user", "content": user_input}],
        model="gpt-4o-mini"
    )
    return result.choices[0].message.content

pipeline("Hello, world!")
# Out:  Hello there! How can I assist you today?

'Hello! How can I assist you today?'

Create a chain using LCEL syntax.

In [38]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Instantiate the ChatOpenAI model.
model = ChatOpenAI()
# Create a prompt template that asks for jokes on a given topic.
prompt = PromptTemplate.from_template("Describe the {topic} in 3 sentences.")
# Connect the prompt and model to create a conversation chain.
chain = prompt | model | StrOutputParser()

## stream: real-time output

This function uses the ```chain.stream``` method to create a stream of data for a given topic, iterating over it and immediately outputting the ```content``` of each piece of data. 
The ```end=""``` argument disables newlines after output, and the ```flush=True``` argument causes the output buffer to be emptied immediately.

In [39]:
# Use the chain.stream method to create a stream of data for a given topic, iterating over it and immediately outputting the content of each piece of data. 
for token in chain.stream({"topic": "multimodal"}):
    # Output the content of each piece of data without newlines.
    print(token, end="", flush=True)

# example output 
# The multimodal approach involves using multiple modes of communication, such as visual, auditory, and kinesthetic, to enhance learning and understanding. By incorporating different sensory inputs, learners are able to engage with material in a more holistic and immersive way. This approach is especially effective in catering to diverse learning styles and preferences.

Multimodal refers to the use of multiple modes of communication, such as visual, auditory, and kinesthetic, to convey information. It emphasizes the importance of engaging multiple senses to enhance understanding and retention of information. By incorporating various modes of communication, multimodal approaches can cater to diverse learning styles and preferences.

## Invoke

The ```invoke``` method of a ```chain``` object takes a topic as an argument and performs processing on that topic.






In [4]:
# Call the invoke method of the chain object, passing a dictionary with the topic 'ChatGPT'.
chain.invoke({"topic": "ChatGPT"})

'ChatGPT is an AI-powered chatbot that uses natural language processing to engage in conversations with users. It can provide information, answer questions, and generate responses based on the input it receives. ChatGPT is constantly learning and improving its capabilities through machine learning algorithms.'

## batch: unit execution

The function ```chain.batch``` takes a list containing multiple dictionaries as arguments and performs batch processing using the values of the ```topic``` key in each dictionary.

In [ ]:
# Call a function to batch process a given list of topics
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

['ChatGPT is an AI-powered chatbot that can engage in natural, fluid conversations with users. It uses advanced natural language processing algorithms to understand and respond to a wide range of topics. ChatGPT is designed to provide personalized and helpful responses to user inquiries, creating a seamless and interactive chat experience.',
 'Instagram is a popular social media platform where users can share photos and videos with their followers. The app also offers features like filters, stories, and direct messaging to enhance the user experience. Many influencers, celebrities, and brands use Instagram to connect with their audience and showcase their content.']

You can use the ```max_concurrency``` parameter to set the number of concurrent requests.

The ```config``` dictionary uses the ```max_concurrency``` key to set the maximum number of operations that can be processed concurrently. Here, it is set to process up to three jobs concurrently.

In [6]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "multimodal"},
        {"topic": "programming"},
        {"topic": "machineLearning"},
    ],
    config={"max_concurrency": 3},
)

['ChatGPT is an AI-powered chatbot that uses natural language processing to engage in conversations with users. It is trained on a diverse range of topics and can provide helpful and informative responses. ChatGPT is constantly learning and improving its ability to understand and generate human-like conversations.',
 'Instagram is a popular social media platform where users can share photos and videos with their followers. It allows users to apply filters and edit their posts before sharing them. Users can also engage with others by liking, commenting, and sharing posts.',
 'The multimodal approach involves using multiple modes of communication, such as visual, auditory, and kinesthetic, to convey information. This approach recognizes that individuals have different learning styles and preferences, and aims to cater to a variety of sensory inputs. By incorporating multiple modes of communication, the multimodal approach can enhance understanding, engagement, and retention of informatio

## async stream

The function ```chain.astream``` creates an asynchronous stream and processes messages for a given topic asynchronously.

It uses an asynchronous for loop (```async for```) to sequentially receive messages from the stream, and the print function to immediately print the contents of the messages (```s.content```). ```end=""``` disables line wrapping after printing, and ```flush=True``` forces the output buffer to be emptied to ensure immediate printing.


In [40]:
# Use an asynchronous stream to process messages in the 'YouTube' topic.
async for token in chain.astream({"topic": "YouTube"}):
    # Print the message content. Outputs directly without newlines and empties the buffer.
    print(token, end="", flush=True)

YouTube is a popular video-sharing platform where users can upload, watch, and interact with videos on a wide variety of topics. It has become a global phenomenon, with billions of users accessing content from around the world every day. The platform also offers opportunities for content creators to monetize their videos through advertising and sponsorships.

## async invoke

The ```ainvoke``` method of a ```chain``` object performs an operation asynchronously with the given arguments. Here, we are passing a dictionary with a key named ```topic``` and a value named ```NVDA``` (NVIDIA's ticker) as arguments. This method can be used to asynchronously request processing for a specific topic.

In [47]:
# Handle the 'NVDA' topic by calling the 'ainvoke' method of the asynchronous chain object.
my_process = chain.ainvoke({"topic": "NVDA(ticker)"})

In [48]:
# Wait for the asynchronous process to complete.
await my_process

"NVDA is the ticker symbol for Nvidia Corporation, a leading technology company known for its graphics processing units (GPUs) and semiconductor products. The company's products are widely used in gaming, artificial intelligence, and data centers. NVDA has seen significant growth in recent years due to the increasing demand for its products in various industries."

## async batch

The function ```abatch``` batches a series of actions asynchronously.

In this example, we are using the ```abatch``` method of the ```chain``` object to asynchronously process actions on ```topic``` .

The ```await``` keyword is used to wait for those asynchronous tasks to complete.

In [52]:
# Performs asynchronous batch processing on a given topic.
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

In [53]:
# Wait for the asynchronous batch process to complete.
await my_abatch_process

['YouTube is a popular video-sharing platform where users can watch, upload, and share videos on a wide range of topics. It offers a diverse range of content, from music videos and tutorials to vlogs and educational videos. Users can also subscribe to channels, like videos, and leave comments to engage with the YouTube community.',
 'Instagram is a social media platform where users can share photos and videos with their followers. The app allows users to apply filters and edit their photos before posting. Users can also interact with each other by liking, commenting, and messaging.',
 'Facebook is a social media platform that allows users to connect with friends and family, share photos and updates, and follow news and events. Users can create personal profiles, join groups, and engage in conversations through comments and likes. The platform also offers advertising opportunities for businesses to reach a larger audience.']

## Parallel

Let's take a look at how the LangChain Expression Language supports parallel requests. For example, when you use ```RunnableParallel``` (often written in dictionary form), you execute each element in parallel.

Here's an example of running two tasks in parallel using the ```RunnableParallel``` class in the ```langchain_core.runnables``` module.

Create two chains (```chain1```, ```chain2```) that use the ```ChatPromptTemplate.from_template``` method to get the capital and area for a given ```country```.

These chains are connected via the ```model``` and pipe (```|```) operators, respectively. Finally, we use the ```RunnableParallel``` class to combine these two chains with the keys ```capital``` and ```area``` to create a ```combined``` object that can be run in parallel.

In [23]:
from langchain_core.runnables import RunnableParallel

# Create a chain that asks for the capital of {country}.
chain1 = (
    PromptTemplate.from_template("What is the capital of {country}?")
    | model
    | StrOutputParser()
)

# Create a chain that asks for the area of {country}.
chain2 = (
    PromptTemplate.from_template("What is the area of {country}?")
    | model
    | StrOutputParser()
)

# Create a parallel execution chain that generates the above two chains in parallel.
combined = RunnableParallel(capital=chain1, area=chain2)

The ```chain1.invoke()``` function calls the ```invoke``` method of the ```chain1``` object.

As an argument, it passes a dictionary with the value ```Canada``` in the key named ```country```.

In [24]:
# Run chain1 .
chain1.invoke({"country": "Canada"})

'The capital of Canada is Ottawa.'

Call ```chain2.invoke()```, this time passing a different country, the ```United States```, for the country key.

In [25]:
# Run chain2 .
chain2.invoke({"country": "USA"})

'The total area of the United States is approximately 9.8 million square kilometers (3.8 million square miles).'

The ```invoke``` method of the ```combined``` object performs the processing for the given ```country```.

In this example, the topic ```USA``` is passed to the ```invoke``` method to run.

In [28]:
# Run a parallel execution chain.
combined.invoke({"country": "USA"})

{'capital': 'The capital of the United States of America is Washington, D.C.',
 'area': 'The total land area of the United States is approximately 3.8 million square miles.'}

## Parallelism in batches

Parallelism can be combined with other executable code. Let's try using parallelism with batch.

The ```chain1.batch``` function takes a list containing multiple dictionaries as an argument, and processes the values corresponding to the "topic" key in each dictionary. In this example, we're batch processing two topics, "Canada" and "United States".

In [30]:
# Perform batch processing.
chain1.batch([{"country": "Canada"}, {"country": "USA"}])


['The capital of Canada is Ottawa.',
 'The capital of the United States is Washington, D.C.']

The ```chain2.batch``` function takes in multiple dictionaries as a list and performs batch processing.

In this example, we request processing for two countries, ```Canada``` and the ```United States```.

In [31]:
# Perform batch processing.
chain2.batch([{"country": "Canada"}, {"country": "USA"}])


['The total land area of Canada is approximately 9.98 million square kilometers.',
 'The total land area of the United States is approximately 3.8 million square miles.']

The combined.batch function is used to process the given data in batches. 

In this example, it takes a list containing two dictionary objects as arguments and batches data for two countries, Canada and the United States, respectively.

In [32]:
# Processes the given data in batches.
combined.batch([{"country": "Canada"}, {"country": "USA"}])


[{'capital': 'The capital of Canada is Ottawa.',
  'area': 'The total land area of Canada is approximately 9.98 million square kilometers.'},
 {'capital': 'The capital of the United States of America is Washington, D.C.',
  'area': 'The total land area of the United States is approximately 3.8 million square miles (9.8 million square kilometers).'}]